In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import cluster, datasets

#Import pytorch:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset, random_split
import torchvision
import torchvision.transforms as transforms
import time

In [2]:
#!git clone https://github.com/prlz77/resnext.pytorch #original checkout
!git clone https://github.com/anburger/DUDeepLearningExam.git #custom

Cloning into 'DUDeepLearningExam'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 45 (delta 13), reused 41 (delta 9), pack-reused 0
Unpacking objects: 100% (45/45), 7.01 MiB | 13.59 MiB/s, done.


In [3]:
#cd resnext.pytorch
import os
os.chdir('DUDeepLearningExam/resnext.pytorch')
#cd DUDeepLearningExam/resnext.pytorch
!pwd

/content/DUDeepLearningExam/resnext.pytorch


Request GPU in notebook setting.

In [4]:
#First attempt to train on flowers-102 dataset with default parameters (adjust number of epochs, otherwise, training takes too long)
time_start = int(time.time())
!python train_AB.py dataset 'flowers-102' -s ./snapshots --log ./logs --ngpu 1 --learning_rate 0.05 -b 128 -e 50
time_end = int(time.time())

100% 344862509/344862509 [00:12<00:00, 28471458.00it/s]
Extracting data/flowers-102/102flowers.tgz to data/flowers-102
100% 502/502 [00:00<00:00, 1947771.15it/s]
100% 14989/14989 [00:00<00:00, 43238254.92it/s]
/content/DUDeepLearningExam/resnext.pytorch/models/model.py:97: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.classifier.weight)
/content/DUDeepLearningExam/resnext.pytorch/models/model.py:102: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.state_dict()[key], mode='fan_out')
CifarResNeXt(
  (conv_1_3x3): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (stage_1): Sequential(
    (stage_1_bottleneck_0): ResNeXtBottleneck(
      (conv_reduce): Conv2d(64, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn_reduce):

In [8]:
print('Total time for training : {} s'.format(time_end - time_start))

Total time for training : 3024 s


In [12]:
!python test_AB.py  dataset 'flowers-102' --ngpu 1 --load ./snapshots/model.pytorch --test_bs 128 

Files already downloaded and verified
/content/DUDeepLearningExam/resnext.pytorch/models/model.py:97: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.classifier.weight)
/content/DUDeepLearningExam/resnext.pytorch/models/model.py:102: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.state_dict()[key], mode='fan_out')
{'base_width': 64, 'batch_size': 128, 'cardinality': 8, 'data_path': 'dataset', 'dataset': 'flowers-102', 'depth': 29, 'load': './snapshots/model.pytorch', 'log': './', 'ngpu': 1, 'prefetch': 2, 'test': False, 'test_bs': 128, 'widen_factor': 4, 'test_loss': 4.498557908194406, 'test_accuracy': tensor(0.0768, device='cuda:0')}
Test error rate : 0.9232395887374878


The model is able to archive a test accuracy (correct classifications / N(data)) of 8.15% (7.7% in a second run). This model has a depth of 29, a cardinality of 8 and a width of 4.
The total execution time is between 55 and 56min for 50 epochs. The number of epochs has been chosen as the loss does not decrease any more after that very much and is a trade-off because of the limited resources (gpu).

In [ ]:
#Download model to be able to do tests locally
#from google.colab import files
#files.download('snapshots/model.pytorch') 
#files.download('logs/log.txt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
#Run plotting scripts
!python utils/plot_log.py

Figure(640x480)
Figure(640x480)
Figure(640x480)


Now, change hyper-parameters. The most important parameter in the ResNeXt model is the cardinality. Decrease to 2 and check effect.

In [14]:
time_start = int(time.time())
!python train_AB.py dataset 'flowers-102' -s ./snapshots --log ./logs --ngpu 1 --learning_rate 0.05 -b 128 -e 50 --cardinality 2
time_end = int(time.time())

/content/DUDeepLearningExam/resnext.pytorch/models/model.py:97: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.classifier.weight)
/content/DUDeepLearningExam/resnext.pytorch/models/model.py:102: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.state_dict()[key], mode='fan_out')
CifarResNeXt(
  (conv_1_3x3): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (stage_1): Sequential(
    (stage_1_bottleneck_0): ResNeXtBottleneck(
      (conv_reduce): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn_reduce): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv_conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2, bias=False)
      (bn): BatchNo

In [15]:
print('Total time for training : {} s'.format(time_end - time_start))

Total time for training : 2420 s


In [ ]:
#Download model snapshot and log for local tests and plotting
#from google.colab import files
#files.download('snapshots/model.pytorch') 
#files.download('logs/log.txt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
!python test_AB.py  dataset 'flowers-102' --ngpu 1 --load ./snapshots/model.pytorch --test_bs 128 --cardinality 2

Files already downloaded and verified
/content/DUDeepLearningExam/resnext.pytorch/models/model.py:97: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.classifier.weight)
/content/DUDeepLearningExam/resnext.pytorch/models/model.py:102: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.state_dict()[key], mode='fan_out')
{'base_width': 64, 'batch_size': 128, 'cardinality': 2, 'data_path': 'dataset', 'dataset': 'flowers-102', 'depth': 29, 'load': './snapshots/model.pytorch', 'log': './', 'ngpu': 1, 'prefetch': 2, 'test': False, 'test_bs': 128, 'widen_factor': 4, 'test_loss': 4.236533315814271, 'test_accuracy': tensor(0.0816, device='cuda:0')}
Test error rate : 0.918360710144043


In [17]:
!python utils/plot_log.py

Figure(640x480)
Figure(640x480)
Figure(640x480)


The model has an accuracy of 8.16% and is more or less equal with the accuracy observed with Cardinality of 8. The training takes 36 minutes.

Now, check whether can increase the model performance by modifying the hyper-parameters: increase the learning rate (lr = 0.05 --> 0.1).

In [ ]:
time_start2 = int(time.time())
!python train_AB.py dataset 'flowers-102' -s ./snapshots --log ./logs --ngpu 1 --learning_rate 0.1 -b 128 -e 50 --cardinality 2
time_end2 = int(time.time())

/content/resnext.pytorch/models/model.py:97: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.classifier.weight)
/content/resnext.pytorch/models/model.py:102: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.state_dict()[key], mode='fan_out')
CifarResNeXt(
  (conv_1_3x3): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (stage_1): Sequential(
    (stage_1_bottleneck_0): ResNeXtBottleneck(
      (conv_reduce): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn_reduce): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv_conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2, bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, aff

In [ ]:
print('Total time for training : {} s'.format(time_end2 - time_start2))

Total time for training : 2122 s


In [ ]:
#Download the model snapshot
#from google.colab import files
#files.download('snapshots/model.pytorch') 
#files.download('logs/log.txt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!python test_AB.py  dataset 'flowers-102' --ngpu 1 --load ./snapshots/model.pytorch --test_bs 128 --cardinality 2

Files already downloaded and verified
/content/resnext.pytorch/models/model.py:97: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.classifier.weight)
/content/resnext.pytorch/models/model.py:102: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.state_dict()[key], mode='fan_out')
Prediction : tensor([93, 57, 47,  0, 66,  0, 57,  0, 90,  0, 60, 59,  0, 63, 54, 29, 59,  0,
        26, 85, 98,  6, 57, 60,  6, 29, 66, 29, 85, 98, 57,  1, 47, 57, 60, 60,
        69, 57, 57, 29, 57, 90, 60, 26, 57, 47, 37, 20, 19, 57,  1,  0, 14, 60,
         0, 47, 37, 57, 71, 20, 97, 75, 19, 54, 60, 65, 97, 60, 30, 59, 65, 26,
        26, 75, 26, 97, 75, 26, 66, 20, 59, 98, 26, 57, 37, 60,  6, 14, 71, 71,
         6, 66,  0, 88,  0,  3, 87,  8,  0, 65, 60, 28, 66, 37, 19,  6, 49, 71,
         1, 85, 22, 57, 54, 26, 90, 29, 26, 26, 26, 26, 26, 88, 88, 26, 26, 26,
        

In [ ]:
#This training is now done with training and test dataset exchanged
#time_start3 = int(time.time())
#!python train_AB.py dataset 'flowers-102' -s ./snapshots --log ./logs --ngpu 1 --learning_rate 0.05 -b 128 -e 50 --cardinality 2
#time_end3 = int(time.time())

/content/resnext.pytorch/models/model.py:97: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.classifier.weight)
/content/resnext.pytorch/models/model.py:102: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.state_dict()[key], mode='fan_out')
CifarResNeXt(
  (conv_1_3x3): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (stage_1): Sequential(
    (stage_1_bottleneck_0): ResNeXtBottleneck(
      (conv_reduce): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn_reduce): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv_conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2, bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, aff

In [ ]:
#print('Total time for training : {} s'.format(time_end3 - time_start3))

Total time for training : 2216 s


In [ ]:
#!python test_AB.py  dataset 'flowers-102' --ngpu 1 --load ./snapshots/model.pytorch --test_bs 128 --cardinality 2

Files already downloaded and verified
/content/resnext.pytorch/models/model.py:97: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.classifier.weight)
/content/resnext.pytorch/models/model.py:102: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.state_dict()[key], mode='fan_out')
Prediction : tensor([36, 36, 85, 35, 87, 93, 45, 83, 21, 76, 87, 77, 72, 72, 42, 76, 87, 74,
        22, 59, 76, 74, 75, 72, 87, 64, 74, 36, 74, 45, 87, 60, 72, 97, 77, 76,
        85, 76, 87, 76, 88, 45, 45, 45, 64, 72, 76, 50, 52, 49, 74, 45, 58, 75,
        72, 43, 60, 45, 45, 50, 72, 50, 74, 74, 88, 74, 36, 74, 72, 90, 45, 45,
        76, 55, 75, 36, 76, 45, 58, 74, 76, 76, 76, 97, 75, 56, 76, 97, 72, 75,
        73, 76, 45, 36, 72, 74, 59, 76, 18, 45, 43, 64, 76, 76, 80, 76, 59, 76,
        50, 73, 36, 87, 74, 87, 11, 49, 11, 50, 11, 47, 74, 73, 76, 22, 35, 50,
        

In [ ]:
#Download model
#from google.colab import files
#files.download('snapshots/model.pytorch') 
#files.download('logs/log.txt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>